In [1]:
import json
import pandas as pd
import psycopg2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random
import os

In [2]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [4]:
def load_place(db):
  cursor= db.cursor()
  
  load_place_sql= f'SELECT place_id,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster FROM places_analysis'
  cursor.execute(load_place_sql)
  result = cursor.fetchall()
  df = pd.DataFrame(result)
  df.columns = ['place_id','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster']
  df = df.sort_values(by='place_id',ascending=True)
  return df

In [3]:
def calculate_taste(user_id,db):
  print("calculate_tast...")
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM places_analysis WHERE place_id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    #print('place : ',place,"\tnow : ",place_theme)

  avg_theme_score = theme_score / len(survey_result)
  user_taste = pd.DataFrame(avg_theme_score.reshape(1,-1))
  user_taste.columns = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']
  print("calculate_tast DONE!!")
  return user_taste

In [29]:
def find_places_in_travel_note(travel_note_id, db):
  cursor= db.cursor()
  sql = f"select places from course where travel_note_id={travel_note_id}"
  cursor.execute(sql)
  result = cursor.fetchall()
  for places in result:
    print(places)
  places = []
  for path in result:
    places += path[0]
  return places

In [30]:
def recommend_place(user_id,travel_note_id):
  
  top = 50
  db = load_db()
  place = load_place(db)
  user_taste = calculate_taste(user_id,db)
  places_in_course = find_places_in_travel_note(travel_note_id, db)
  tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']
  

  #entire tag
  print(user_taste)
  entire_similarity = cosine_similarity(user_taste, place[tag])
  entire_similarity = entire_similarity[0]
      
  print('entire similarity : ',entire_similarity)
  
  result_index = entire_similarity.argsort()
  
  print("result_index : ",result_index)


  for i in range(-1,-4,-1):
    now_idx = result_index[i]
    sim = entire_similarity[now_idx]
    print("now place ",place.iloc[now_idx])
    print("similarity : ",sim,"df index : ",now_idx)
    

  top_N_result = result_index[-1:-(101):-1]
  print("top_N_similart_result",top_N_result[:10])
  
  result = place.iloc[top_N_result].sort_values(by='place_id',ascending=True)
  result = result.iloc[:top]
  print("result.iloc[:top]")
  print(result)

  result = random.sample(list(result['place_id']), 2) 
  # if course에 있는거랑 곂치면 다시 뽑자!!!
  while set(result).intersection(places_in_course) != set():
    result = random.sample(list(result['place_id']), 2)

  places_in_course += result

  
  print("result **id** list",result)
  
  print("*"*50)

  #6개 태그만 적용하기
  #2개를 뽑아서 없앤다음, 적용하자.4개중 nature, exciting, culture, day
  six_tag = tag.copy()
  picking_list = ['nature', 'exciting', 'sea', 'day']
  randomly_chosen_two_tags = random.sample(picking_list,2)
  for tag in randomly_chosen_two_tags:
    six_tag.remove(tag) #뽑힌 2개는 지워주기

  user_taste_six = user_taste[six_tag]
  print("user taste six : ",user_taste_six)
  six_tag_similarity = cosine_similarity(user_taste_six, place[six_tag])
  six_tag_similarity = six_tag_similarity[0]
  
  print('six similarity : ',six_tag_similarity)
  
  six_result_index = six_tag_similarity.argsort()
  
  print("six result_index : ",six_result_index)

  for i in range(-1,-4,-1):
    now_idx = six_result_index[i]
    sim = six_tag_similarity[now_idx]
    print("now place ",place.iloc[now_idx])
    print("similarity : ",sim,"df index : ",now_idx)
    
  top_N_result_six = six_result_index[-1:-101:-1]
  print("top_N_result(only six tag)",top_N_result_six[:10])
  
  result_six = place.iloc[top_N_result_six].sort_values(by='place_id',ascending=True)
  result_six = result_six.iloc[:top]
  print("result _ six")
  print(result_six)
  
  result_six = random.sample(list(result_six['place_id']), 2) # if 위에서 뽑은거랑 곂치면 다시 뽑자!!!
  while set(result_six).intersection(places_in_course) != set():
    result_six = random.sample(list(result_six['place_id']), 2)

  
  final_result = result+result_six

  return final_result

In [27]:
db = load_db()
path = find_places_in_travel_note(311,db)

([18, 46],)
([223, 80, 0],)


In [31]:
final_result = recommend_place(user_id=184,travel_note_id=311)

calculate_tast...
[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]
calculate_tast DONE!!
([18, 46],)
([223, 80, 0],)
   nature  outdoor  fatigue  sea  walking  exciting  day  culture
0     2.0      2.5      1.4  1.6      2.2       1.3  2.8      0.1
entire similarity :  [0.82417379 0.93669009 0.89572696 ... 0.42084073 0.55776359 0.84998234]
result_index :  [449 450 513 ... 607 516 440]
now place  place_id    441
nature        2
outdoor       2
fatigue       1
sea           1
walking       2
exciting      1
day           2
culture       0
cluster       8
Name: 453, dtype: int64
similarity :  0.990056184533482 df index :  440
now place  place_id    517
nature        2
outdoor       2
fatigue       1
sea           1
walking       2
exciting      1
day           2
culture       0
cluster       8
Name: 526, dtype: int64
similarity :  0.990056184533482 df index :  516
now place  place_id    608
nature        2
outdoor       2
fatigue       1
sea           1
walking       2
exciting      1
day          

In [32]:
final_result

[92, 270, 469, 179]

In [21]:
path

[([18, 46],), ([223, 80, 0],)]

In [25]:
result = []
for i in path:
  print(i[0])
  result += i[0]
result

[18, 46]
[223, 80, 0]


[18, 46, 223, 80, 0]